In [20]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

In [21]:
MAX_NUM_WORDS = 20000       # max vocab size
MAX_SEQUENCE_LENGTH = 300   # max words per input
EMBEDDING_DIM = 100         # embedding vector size
BATCH_SIZE = 32
EPOCHS = 20

In [22]:
df = pd.read_csv("Combined Data.csv")  # <-- your CSV file
print(df.head())

                                                text sentiment
0                                         oh my gosh   Anxiety
1  trouble sleeping, confused mind, restless hear...   Anxiety
2  All wrong, back off dear, forward doubt. Stay ...   Anxiety
3  I've shifted my focus to something else but I'...   Anxiety
4  I'm restless and restless, it's been a month n...   Anxiety


In [23]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)   # remove urls
    text = re.sub(r"[^a-z\s]", " ", text) # keep only letters
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["clean_text"] = df["text"].apply(clean_text)

In [24]:
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["sentiment"])
num_classes = len(label_encoder.classes_)
print("Classes:", label_encoder.classes_)

Classes: ['Anxiety' 'Bipolar' 'Depression' 'Normal' 'Personality disorder' 'Stress'
 'Suicidal' nan]


In [25]:
y = to_categorical(df["label"], num_classes=num_classes)


In [27]:
X_train, X_temp, y_train, y_temp = train_test_split(
    df["clean_text"], y, test_size=0.2, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42
)


In [28]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

In [29]:
seq_train = tokenizer.texts_to_sequences(X_train)
seq_val = tokenizer.texts_to_sequences(X_val)
seq_test = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(seq_train, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
X_val_pad = pad_sequences(seq_val, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
X_test_pad = pad_sequences(seq_test, maxlen=MAX_SEQUENCE_LENGTH, padding="post")

In [30]:
model = Sequential()
model.add(Embedding(input_dim=MAX_NUM_WORDS, 
                    output_dim=EMBEDDING_DIM, 
                    input_length=MAX_SEQUENCE_LENGTH,
                    name="embedding"))
model.add(Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3), 
                       name="bidirectional"))
model.add(Dense(64, activation="relu", name="dense"))
model.add(Dropout(0.5, name="dropout"))
model.add(Dense(num_classes, activation="softmax", name="dense_1"))

model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

# Build the model by calling it with a sample input shape
# This forces Keras to build the model and compute parameter counts
model.build(input_shape=(None, MAX_SEQUENCE_LENGTH))

print(model.summary())

c:\Users\NItesh\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 300, 100)       │     2,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,093,256 (7.99 MB)

 Trainable params: 2,093,256 (7.99 MB)

 Non-trainable params: 0 (0.00 B)

None


In [31]:
early_stop = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)

Epoch 1/20
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 237s 176ms/step - accuracy: 0.5656 - loss: 1.0779 - val_accuracy: 0.6278 - val_loss: 0.8786
Epoch 2/20
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 243s 183ms/step - accuracy: 0.6603 - loss: 0.8213 - val_accuracy: 0.6931 - val_loss: 0.7487
Epoch 3/20
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 251s 189ms/step - accuracy: 0.7288 - loss: 0.6869 - val_accuracy: 0.7266 - val_loss: 0.6905
Epoch 4/20
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 256s 193ms/step - accuracy: 0.7732 - loss: 0.5930 - val_accuracy: 0.7449 - val_loss: 0.6619
Epoch 5/20
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 266s 200ms/step - accuracy: 0.8100 - loss: 0.5037 - val_accuracy: 0.7611 - val_loss: 0.6646
Epoch 6/20
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 258s 194ms/step - accuracy: 0.8407 - loss: 0.4322 - val_accuracy: 0.7613 - val_loss: 0.6678
Epoch 7/20
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 218s 164ms/step - accuracy: 0.8640 - loss: 0.3764 - val_accuracy: 0.7641 - val_loss: 0.6944
Epoch 8/20
1327/1327 ━━━━━━━━━━━━━━━━━━━━ 230s 174ms/step - ac

In [34]:
loss, acc = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}, Test Loss: {loss:.4f}")

model.save("mental_health_model.h5")

Test Accuracy: 0.7482, Test Loss: 1.6591


In [35]:
model = tf.keras.models.load_model(r"C:\Users\Nitesh\OneDrive\Desktop\models\mental_health_model.h5")
def predict_sentiment(text):
    text_clean = clean_text(text)
    seq = tokenizer.texts_to_sequences([text_clean])
    pad = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
    prob = model.predict(pad)[0]
    label_id = np.argmax(prob)
    return label_encoder.classes_[label_id], prob[label_id]

In [49]:
example = "I feel like dying and have no hope."
pred_label, pred_prob = predict_sentiment(example)
print("Predicted:", pred_label, "with probability:", pred_prob)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted: Normal with probability: 0.8804539


In [37]:
import pickle

# Save tokenizer
with open("mental_health_model_tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# Save label encoder
with open("mental_health_model_label_encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

In [50]:
##SImple RNN

In [ ]:
import pandas as pd import numpy as np import re import tensorflow as tf from sklearn.model_selection import train_test_split from sklearn.preprocessing import LabelEncoder from tensorflow.keras.preprocessing.text import Tokenizer from tensorflow.keras.preprocessing.sequence import pad_sequences from tensorflow.keras.utils import to_categorical from tensorflow.keras.models import Sequential from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional from tensorflow.keras.callbacks import EarlyStopping MAX_NUM_WORDS = 20000 # max vocab size MAX_SEQUENCE_LENGTH = 300 # max words per input EMBEDDING_DIM = 100 # embedding vector size BATCH_SIZE = 32 EPOCHS = 20 df = pd.read_csv("Combined Data.csv") # <-- your CSV file print(df.head()) def clean_text(text): text = str(text).lower() text = re.sub(r"http\S+", "", text) # remove urls text = re.sub(r"[^a-z\s]", " ", text) # keep only letters text = re.sub(r"\s+", " ", text).strip() return text df["clean_text"] = df["text"].apply(clean_text) label_encoder = LabelEncoder() df["label"] = label_encoder.fit_transform(df["sentiment"]) num_classes = len(label_encoder.classes_) print("Classes:", label_encoder.classes_) y = to_categorical(df["label"], num_classes=num_classes) X_train, X_temp, y_train, y_temp = train_test_split( df["clean_text"], y, test_size=0.2, stratify=y, random_state=42 ) X_val, X_test, y_val, y_test = train_test_split( X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42 ) tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, oov_token="<OOV>") seq_train = tokenizer.texts_to_sequences(X_train) seq_val = tokenizer.texts_to_sequences(X_val) seq_test = tokenizer.texts_to_sequences(X_test) X_train_pad = pad_sequences(seq_train, maxlen=MAX_SEQUENCE_LENGTH, padding="post") X_val_pad = pad_sequences(seq_val, maxlen=MAX_SEQUENCE_LENGTH, padding="post") X_test_pad = pad_sequences(seq_test, maxlen=MAX_SEQUENCE_LENGTH, padding="post")tokenizer.fit_on_texts(X_train) model = Sequential() model.add(Embedding(input_dim=MAX_NUM_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH, name="embedding")) model.add(Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3), name="bidirectional")) model.add(Dense(64, activation="relu", name="dense")) model.add(Dropout(0.5, name="dropout")) model.add(Dense(num_classes, activation="softmax", name="dense_1")) model.compile( loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"] ) # Build the model by calling it with a sample input shape # This forces Keras to build the model and compute parameter counts model.build(input_shape=(None, MAX_SEQUENCE_LENGTH)) print(model.summary()) early_stop = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True) history = model.fit( X_train_pad, y_train, validation_data=(X_val_pad, y_val), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stop], verbose=1 ) loss, acc = model.evaluate(X_test_pad, y_test, verbose=0) print(f"Test Accuracy: {acc:.4f}, Test Loss: {loss:.4f}") model.save("mental_health_model.h5")